In [ ]:
# Imports
import os
import json
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import re


# For deep learning
import torch
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.ops import box_iou

from torchmetrics.detection.mean_ap import MeanAveragePrecision
from sklearn.metrics import precision_recall_fscore_support
import numpy as np


# For augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Own package imports
os.chdir('/home/naro/projects/Rumex')

from config.paths_config import *
from config.config import *
from data.data_inspection import *
from data.augmentation import *
from data.dataset import *
from utils.viz_utils import *
from utils.data_utils import *
from models.model_factory import *
from scripts.evaluate import *
from scripts.train import *
from scripts.inference import predict_and_visualize_image, load_best_model
from utils.fiftyone_utils import *
from tuning.hyperparameters_tuning import *

VIZ = False

In [ ]:
# Initialize PathsConfig
pathconfig ={
    "dataset_name": "haldennord09",
    "darwin_root": "/home/naro/.darwin/datasets/digital-production",
    "dataset_version": "latest",
    "extension": 'png',
    "models_dir": '/home/naro/projects/Rumex/artifacts/models'
} 

paths_config = PathsConfig(**pathconfig)

# Initialize DataVerifier
data_verifier = DataVerifier(
    img_dir=paths_config.img_dir,
    annotations_dir=paths_config.annotations_dir,
    train_split_file=paths_config.train_split_file,
    test_split_file=paths_config.test_split_file,
    val_split_file=paths_config.val_split_file,
    extension=paths_config.extension
)

# Verify data
train_annotations, test_annotations, val_annotations = data_verifier.check_directory_contents()

# Initialize ImageProcessor
image_processor = ImagesClassesInspector(
    img_dir=paths_config.img_dir,
    annotations_dir=paths_config.annotations_dir
)

# Get image and annotation lists
train_images = data_verifier.get_image_files(train_annotations)
val_images = data_verifier.get_image_files(val_annotations)
test_images = data_verifier.get_image_files(test_annotations)

# Get image sizes
image_files = os.listdir(paths_config.img_dir)
train_sizes = image_processor.get_image_sizes(image_files)

# Get classes
annotation_files = train_annotations + test_annotations + val_annotations
classes = image_processor.get_classes(annotation_files)
print("\nClasses in the dataset:")
print(classes)

class_map = {name: idx + 1 for idx, name in enumerate(classes)}
print("\nThe created class map:")
print(class_map)

# Get image size stats
min_size, max_size = image_processor.get_image_size_stats(image_files)
print(f"Smallest image size: {min_size}")
print(f"Largest image size: {max_size}")

w_min, h_min = min_size
print(f"Width of smallest image: {w_min}")
print(f"Height of smallest image: {h_min}")

# Initialize AugmentationConfig
augmentation_config = AugmentationConfig(height=h_min, width=w_min)

# Get transforms
train_transform = augmentation_config.get_train_transform()
valid_transform = augmentation_config.get_valid_transform()

# Print transform configurations
print("Training transforms:")
print(train_transform)
print("\nValidation transforms:")
print(valid_transform)

train_loader, val_loader, test_loader = create_data_loaders(
    img_dir=paths_config.img_dir,
    annotation_dir=paths_config.annotations_dir,
    train_images=train_images,
    train_annotations=train_annotations,
    val_images=val_images,
    val_annotations=val_annotations,
    test_images=test_images,
    test_annotations=test_annotations,
    train_transform=train_transform,
    valid_transform=valid_transform,
    class_map=class_map
)

# Print the number of samples in each dataset
print(f"Number of samples in training dataset: {len(train_loader.dataset)}")
print(f"Number of samples in validation dataset: {len(val_loader.dataset)}")
print(f"Number of samples in test dataset: {len(test_loader.dataset)}")


In [ ]:
# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define number of classes (update this for your dataset)
num_classes = 2  # e.g., background + bird

# Initialize a Faster R-CNN model with ResNet50 backbone
model_config = {
    'model_name': 'fasterrcnn',
    'backbone_name': 'resnet50',
    'num_classes': num_classes,
    'device': device,
    'weights': 'COCO_V1',
    'train_backbone': False
}

model = init_model(**model_config)

model_path = "/home/naro/projects/Rumex/mlruns/260316289658278492/b6c988fecd6b4b6898a199a1064a1b3b/artifacts/model/data/model.pth"  # Update with your model path
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()


In [ ]:
# Evaluate it using the map50 function

In [ ]:
# Implement a new evaluation function here, then deploy it to the pipeline